In [9]:
#import pandas and sqlite
import pandas as pd
import sqlite3 as sl

In [10]:
#connecct to database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [11]:
#update counties table with most recent data
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE IF EXISTS counties')
c.execute('CREATE TABLE counties (date datetime, county str, state str, fips str, cases int, deaths int, confirmed_cases int, confirmed_deaths int, probable_cases int, probable_deaths int)')
conn.commit()

counties.to_sql('counties',conn, if_exists='replace')

In [12]:
#create new table for regions

regions = pd.read_csv('Untitled.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE IF EXISTS regions')
c.execute('CREATE TABLE regions (state str, census_bureau_region str, census_bureau_division str, standard_federal_region str, circuit_court_appeals str, bureau_economic_analysis str)')
conn.commit()

regions.to_sql('regions',conn, if_exists='replace')

/opt/anaconda3/envs/amanda/lib/python3.8/site-packages/pandas/core/generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [13]:
#confirm tables present
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

deaths
mask_use
states
us
election
counties
regions


In [14]:
df = pd.read_sql('''select * from counties where date = '2020-09-07' limit 15''',conn)
df

,date,county,state,fips,cases,deaths
0,2020-09-07,Autauga,Alabama,1001.0,1504,24
1,2020-09-07,Baldwin,Alabama,1003.0,4730,46
2,2020-09-07,Barbour,Alabama,1005.0,778,7
3,2020-09-07,Bibb,Alabama,1007.0,583,9
4,2020-09-07,Blount,Alabama,1009.0,1384,12
5,2020-09-07,Bullock,Alabama,1011.0,568,14
6,2020-09-07,Butler,Alabama,1013.0,892,38
7,2020-09-07,Calhoun,Alabama,1015.0,2877,39
8,2020-09-07,Chambers,Alabama,1017.0,1063,41
9,2020-09-07,Cherokee,Alabama,1019.0,452,12


In [15]:
df = pd.read_sql('''select * from counties where state = 'Illinois' and county = 'Cook' and date >= '2020-09-07' order by date limit 30''',conn)
df

,date,county,state,fips,cases,deaths
0,2020-09-07,Cook,Illinois,17031.0,131622,5097
1,2020-09-08,Cook,Illinois,17031.0,132079,5098
2,2020-09-09,Cook,Illinois,17031.0,132401,5100
3,2020-09-10,Cook,Illinois,17031.0,132966,5103
4,2020-09-11,Cook,Illinois,17031.0,133686,5113
5,2020-09-12,Cook,Illinois,17031.0,134352,5128
6,2020-09-13,Cook,Illinois,17031.0,134801,5133
7,2020-09-14,Cook,Illinois,17031.0,135274,5134
8,2020-09-15,Cook,Illinois,17031.0,135690,5137
9,2020-09-16,Cook,Illinois,17031.0,136246,5146


In [16]:
df = pd.read_sql('''SELECT date, state, county, cases,
                            cases - LAG(cases,1) OVER (ORDER BY date) as 'new_cases'
                    FROM counties
                    WHERE state = 'Illinois' AND county = 'Cook' AND date >= '2020-09-01'
                    ORDER BY date''',conn)

df

,date,state,county,cases,new_cases
0,2020-09-01,Illinois,Cook,126992,NaN
1,2020-09-02,Illinois,Cook,127616,624.0
2,2020-09-03,Illinois,Cook,128012,396.0
3,2020-09-04,Illinois,Cook,129545,1533.0
4,2020-09-05,Illinois,Cook,130616,1071.0
...,...,...,...,...,...
57,2020-10-28,Illinois,Cook,181108,2276.0
58,2020-10-29,Illinois,Cook,183649,2541.0
59,2020-10-30,Illinois,Cook,186405,2756.0
60,2020-10-31,Illinois,Cook,186405,0.0


In [17]:
df = pd.read_sql('''SELECT date, state, county, cases,
                            cases - LAG(cases,1) OVER (ORDER BY date) as 'new_cases',
                            cases - LAG(cases,7) OVER (ORDER BY date) as '7-day_new_cases'
                    FROM counties
                    WHERE state = 'Illinois' AND county = 'Cook' AND date >= '2020-09-01'
                    ORDER BY date''',conn)

df

,date,state,county,cases,new_cases,7-day_new_cases
0,2020-09-01,Illinois,Cook,126992,NaN,NaN
1,2020-09-02,Illinois,Cook,127616,624.0,NaN
2,2020-09-03,Illinois,Cook,128012,396.0,NaN
3,2020-09-04,Illinois,Cook,129545,1533.0,NaN
4,2020-09-05,Illinois,Cook,130616,1071.0,NaN
...,...,...,...,...,...,...
57,2020-10-28,Illinois,Cook,181108,2276.0,12969.0
58,2020-10-29,Illinois,Cook,183649,2541.0,13610.0
59,2020-10-30,Illinois,Cook,186405,2756.0,14968.0
60,2020-10-31,Illinois,Cook,186405,0.0,12588.0


In [18]:
df = pd.read_sql('''SELECT date, state, county, cases,
                            cases - LEAD(cases,1) OVER (ORDER BY date DESC) as 'new_cases',
                            cases - LEAD(cases,7) OVER (ORDER BY date DESC) as '7-day_new_cases'
                    FROM counties
                    WHERE state = 'Illinois' AND county = 'Cook' AND date 
                    ''',conn)

df

,date,state,county,cases,new_cases,7-day_new_cases
0,2020-11-01,Illinois,Cook,193102,6697.0,17516.0
1,2020-10-31,Illinois,Cook,186405,0.0,12588.0
2,2020-10-30,Illinois,Cook,186405,2756.0,14968.0
3,2020-10-29,Illinois,Cook,183649,2541.0,13610.0
4,2020-10-28,Illinois,Cook,181108,2276.0,12969.0
...,...,...,...,...,...,...
278,2020-01-28,Illinois,Cook,1,0.0,NaN
279,2020-01-27,Illinois,Cook,1,0.0,NaN
280,2020-01-26,Illinois,Cook,1,0.0,NaN
281,2020-01-25,Illinois,Cook,1,0.0,NaN


In [19]:
df = pd.read_sql('''SELECT date, state, county, cases, 
                    cases - LAG(cases,1) OVER (PARTITION BY county ORDER BY date) as 'new_cases',
                    cases - LAG(cases,7) OVER (PARTITION BY county ORDER BY date) as '7-day_new_cases'
                    FROM counties
                    WHERE state = 'Illinois' AND date >= '2020-10-15'
                    ORDER BY county, date
                    LIMIT 50''',conn)
df

,date,state,county,cases,new_cases,7-day_new_cases
0,2020-10-15,Illinois,Adams,1515,NaN,NaN
1,2020-10-16,Illinois,Adams,1558,43.0,NaN
2,2020-10-17,Illinois,Adams,1568,10.0,NaN
3,2020-10-18,Illinois,Adams,1616,48.0,NaN
4,2020-10-19,Illinois,Adams,1670,54.0,NaN
5,2020-10-20,Illinois,Adams,1677,7.0,NaN
6,2020-10-21,Illinois,Adams,1743,66.0,NaN
7,2020-10-22,Illinois,Adams,1771,28.0,256.0
8,2020-10-23,Illinois,Adams,1838,67.0,280.0
9,2020-10-24,Illinois,Adams,1892,54.0,324.0


In [20]:
df = pd.read_sql('''WITH cte as (SELECT date, state, county, 
                                cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                                cases as 'cumulative_cases'
                    FROM counties)
                    
                    SELECT date, state, county, new_cases, cumulative_cases,
                    AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7_day_avg_new'
                    FROM cte
                    ORDER BY state, county, date limit 25''',conn)
df

,date,state,county,new_cases,cumulative_cases,7_day_avg_new
0,2020-03-24,Alabama,Autauga,NaN,1,NaN
1,2020-03-25,Alabama,Autauga,3.0,4,3.000000
2,2020-03-26,Alabama,Autauga,2.0,6,2.500000
3,2020-03-27,Alabama,Autauga,0.0,6,1.666667
4,2020-03-28,Alabama,Autauga,0.0,6,1.250000
5,2020-03-29,Alabama,Autauga,0.0,6,1.000000
6,2020-03-30,Alabama,Autauga,1.0,7,1.000000
7,2020-03-31,Alabama,Autauga,0.0,7,0.857143
8,2020-04-01,Alabama,Autauga,3.0,10,0.857143
9,2020-04-02,Alabama,Autauga,0.0,10,0.571429
